In [ ]:
!git clone https://github.com/oscarcapote/MMSBM_library

Cloning into 'MMSBM_library'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 318 (delta 20), reused 34 (delta 14), pack-reused 269
Receiving objects: 100% (318/318), 322.41 KiB | 3.93 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [ ]:
# !rm -r MMSBM_library/

In [ ]:
import pandas as pd
import numpy as np
from numba import jit
import sys, os



import MMSBM_library as sbm

In [ ]:
# Download MovieLens data.
print("Downloading movielens data...")
from urllib.request import urlretrieve
import zipfile

urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


In [ ]:
df_users = pd.read_csv("ml-100k/u.user",sep="|",names=["uid","age","gender","occupation","zip_code"])
df_movies = pd.read_csv("ml-100k/u.item",sep="|",usecols=[0,1,2],names=["mid","title","date"], encoding='latin-1')


In [ ]:
users = sbm.nodes_layer(nodes_info=df_users,K=10,nodes_name="uid")
movies = sbm.nodes_layer(nodes_info=df_movies,K=10,nodes_name="mid")

In [ ]:
for f in range(1,6):

  df_links = pd.read_csv("ml-100k/u{}.base".format(f),sep="\t",names=["uid","mid","rating","time_stamp"])
  ratings = sbm.BiNet(df_links,"rating",nodes_a=users,nodes_b=movies)

  ratings.init_MAP()

  for step in range(100):
    ratings.MAP_step()
    if ratings.converges():
        print("converged!!!")
        break

(10, 10, 5)
(943, 10)
(1682, 10)


IndexError: index 944 is out of bounds for axis 0 with size 943

In [ ]:
movies.theta

In [ ]:
users.df

In [ ]:
ratings.links_training.max(axis=0)

In [ ]:
def omega_comp_arrays(Na,Nb,p_kl,theta,eta,K,L,links_array,links_ratings):
    #new_omega = np.array(omega)
    omega = np.zeros((Na,Nb,K,L))
    for link  in range(len(links_ratings)):
        i = links_array[link][0]
        j = links_array[link][1]
        if i==944:print(i,j)
        if j==944:print(link,i,j)
        rating = links_ratings[link]
        omega[i,j,:,:] = p_kl[:,:,rating]*(np.expand_dims(theta[i,:], axis=1)@np.expand_dims(eta[j,:],axis=0))
        suma = omega[i,j,:,:].sum()
        omega[i,j,:,:] /= suma+1e-16
    return omega


In [ ]:
omega_comp_arrays(len(ratings.nodes_a),len(ratings.nodes_b),ratings.pkl,ratings.nodes_a.theta,ratings.nodes_b.theta,ratings.nodes_a.K,ratings.nodes_b.K,
                               ratings.links_training,ratings.labels_training)

1436 15 944


IndexError: index 944 is out of bounds for axis 0 with size 943

In [ ]:
ratings.pkl[:,:,0]*(np.expand_dims(users.theta[15,:], axis=1)@np.expand_dims(movies.theta[944,:],axis=0))

IndexError: index 944 is out of bounds for axis 0 with size 943

In [ ]:
movies.theta.shape

(943, 10)

In [ ]:
ratings.init_MAP()

(10, 10, 5)
(943, 10)
(1682, 10)
